# Modeling and solving search problems

Consider the following river crossing puzzle:

## Jealous Husbands Problem with an Island

There are n couples: $(𝐻_1,𝑊_1)$, $(𝐻_2,𝑊_2)$, …, $(𝐻_𝑛,𝑊_𝑛)$  where $𝐻_𝑖$ is husband $𝑖$ and $𝑊_𝑖$ is wife $𝑖$.

They need to cross from the left bank of a river to the right bank.

There is a boat that can carry at most two people at a time.

Important: There is an island midway between the left and right banks. The boat can stop at the island.

Rules:

1. Jealousy constraint: No wife may ever be in the presence of another husband (on any shore or the island or in the boat) unless her own husband is also present. Otherwise, fights (and failures) occur.

2. The boat needs at least one person to operate (no remote-controlled trips).

3. The boat can travel between:

            Left bank ↔ Island

            Island ↔ Right bank

4. All people, including the boat, start at the left bank.

Goal: get everyone (husbands and wives) safely to the right bank, obeying all the rules.


## Tasks:

1. Put the problem in a form of a search tree: root, nodes, solution and rules for constructing the childrens.
2. Solve it usinh a classic DFS
3. Solve it using a classic Back-tracking.
4. Solve it using the A* method.  

In [13]:
from typing import Set, Tuple, List
from itertools import combinations
from collections import deque
import heapq

n = 3

HUSBANDS = {f"H{i}" for i in range(1, n + 1)}
WIVES = {f"W{i}" for i in range(1, n + 1)}
ALL_PEOPLE = HUSBANDS | WIVES

BOAT_LOCATIONS = ["left", "island", "right"]

class State:
    def __init__(self, left: Set[str], island: Set[str], right: Set[str], boat_location: str, path: List = None):
        self.left = left
        self.island = island
        self.right = right
        self.boat_location = boat_location
        self.path = path if path is not None else []

    def is_goal(self):
        return self.right == ALL_PEOPLE

    def copy(self):
        return State(set(self.left), set(self.island), set(self.right), self.boat_location, self.path.copy())

    def __eq__(self, other):
        return (
            isinstance(other, State) and
            self.left == other.left and
            self.island == other.island and
            self.right == other.right and
            self.boat_location == other.boat_location
        )

    def __hash__(self):
        return hash((frozenset(self.left), frozenset(self.island), frozenset(self.right), self.boat_location))

    def __repr__(self):
        return f"State(L={self.left}, I={self.island}, R={self.right}, boat={self.boat_location})"

def is_valid_group(group: Set[str]) -> bool:
    for i in range(1, n + 1):
        w, h = f"W{i}", f"H{i}"
        if w in group:
            other_husbands = {x for x in group if x.startswith("H") and x != h}
            if other_husbands and h not in group:
                return False
    return True

def is_valid_state(state: State, boat_passengers: Set[str]) -> bool:
    return (
        is_valid_group(state.left)
        and is_valid_group(state.island)
        and is_valid_group(state.right)
        and is_valid_group(boat_passengers)
    )

def get_adjacent_locations(current: str) -> List[str]:
    if current == "left":
        return ["island"]
    elif current == "island":
        return ["left", "right"]
    elif current == "right":
        return ["island"]
    return []

def generate_children(state: State) -> List[State]:
    children = []
    current_location = state.boat_location
    people_here = getattr(state, current_location)
    possible_passengers = list(combinations(people_here, 1)) + list(combinations(people_here, 2))

    for passengers in possible_passengers:
        passengers = set(passengers)
        for destination in get_adjacent_locations(current_location):
            new_state = state.copy()
            getattr(new_state, current_location).difference_update(passengers)
            getattr(new_state, destination).update(passengers)
            new_state.boat_location = destination
            if is_valid_state(new_state, passengers):
                new_state.path.append((passengers, current_location, destination))
                children.append(new_state)
    return children

initial_state = State(left=set(ALL_PEOPLE), island=set(), right=set(), boat_location="left")

# DFS
def dfs(state: State, visited=None):
    if visited is None:
        visited = set()
    if state.is_goal():
        return state.path
    visited.add(state)
    for child in generate_children(state):
        if child not in visited:
            result = dfs(child, visited)
            if result:
                return result
    return None

def backtracking(state: State, visited: Set[State], path: List[Tuple[Set[str], str, str]]) -> List[Tuple[Set[str], str, str]]:
    if state.is_goal():
        return path

    visited.add(state)

    for child in generate_children(state):
        passengers, src, dst = child.path[-1]
        if child not in visited:
            path.append((passengers, src, dst))  
            result = backtracking(child, visited, path)
            if result:
                return result
            path.pop()  

    return None



def heuristic(state: State):
    return len(ALL_PEOPLE) - len(state.right)

from itertools import count

def a_star(state: State):
    open_set = []
    counter = count()  

    heapq.heappush(open_set, (heuristic(state), 0, next(counter), state))
    visited = set()

    while open_set:
        _, cost, _, current = heapq.heappop(open_set)
        if current in visited:
            continue
        if current.is_goal():
            return current.path
        visited.add(current)
        for child in generate_children(current):
            heapq.heappush(open_set, (
                cost + 1 + heuristic(child),  
                cost + 1,                    
                next(counter),              
                child                       
            ))
    return None



In [3]:
#Task 2
def dfs(state: State, visited=None):
    if visited is None:
        visited = set()
    if state.is_goal():
        return state.path
    visited.add(state)
    for child in generate_children(state):
        if child not in visited:
            result = dfs(child, visited)
            if result:
                return result
    return None

solution = dfs(initial_state)

print("=== DFS Solution ===")
if solution:
    for i, step in enumerate(solution, 1):
        people, frm, to = step
        print(f"{i}. Moved {people} from {frm} to {to}")
    print(f"\nTotal steps: {len(solution)}")
else:
    print("No valid solution found.")

=== DFS Solution ===
1. Moved {'W2', 'W1'} from left to island
2. Moved {'W2'} from island to left
3. Moved {'H1'} from left to island
4. Moved {'W1', 'H1'} from island to right
5. Moved {'H1'} from right to island
6. Moved {'H1'} from island to left
7. Moved {'W2'} from left to island
8. Moved {'W2'} from island to right
9. Moved {'W1'} from right to island
10. Moved {'W1'} from island to left
11. Moved {'W3'} from left to island
12. Moved {'W3'} from island to right
13. Moved {'W2'} from right to island
14. Moved {'W2'} from island to left
15. Moved {'H3'} from left to island
16. Moved {'H3'} from island to right
17. Moved {'H3', 'W3'} from right to island
18. Moved {'H3'} from island to left
19. Moved {'W2'} from left to island
20. Moved {'W3'} from island to left
21. Moved {'W1', 'W3'} from left to island
22. Moved {'W2'} from island to left
23. Moved {'H3', 'H1'} from left to island
24. Moved {'H3', 'W3'} from island to left
25. Moved {'W2', 'H2'} from left to island
26. Moved {'W

In [24]:
def backtracking(state: State, visited: Set[State], path: List[Tuple[Set[str], str, str]]) -> List[Tuple[Set[str], str, str]]:
    if state.is_goal():
        return path

    visited.add(state)

    for child in generate_children(state):
        passengers, src, dst = child.path[-1]
        if child not in visited:
            path.append((passengers, src, dst)) 
            result = backtracking(child, visited, path)
            if result:
                return result
            path.pop() 

    return None

solution = backtracking(initial_state, set(), [])

print("=== Backtracking Solution ===")
if solution:
    for i, (people, frm, to) in enumerate(solution, 1):
        print(f"{i}. Moved {people} from {frm} to {to}")
    print(f"\nTotal steps: {len(solution)}")
else:
    print("No valid solution found.")


=== Backtracking Solution ===
1. Moved {'W2', 'W1'} from left to island
2. Moved {'W2'} from island to left
3. Moved {'H1'} from left to island
4. Moved {'W1', 'H1'} from island to right
5. Moved {'H1'} from right to island
6. Moved {'H1'} from island to left
7. Moved {'W2'} from left to island
8. Moved {'W2'} from island to right
9. Moved {'W1'} from right to island
10. Moved {'W1'} from island to left
11. Moved {'W3'} from left to island
12. Moved {'W3'} from island to right
13. Moved {'W2'} from right to island
14. Moved {'W2'} from island to left
15. Moved {'H3'} from left to island
16. Moved {'H3'} from island to right
17. Moved {'H3', 'W3'} from right to island
18. Moved {'H3'} from island to left
19. Moved {'W2'} from left to island
20. Moved {'W3'} from island to left
21. Moved {'W1', 'W3'} from left to island
22. Moved {'W2'} from island to left
23. Moved {'H3', 'H1'} from left to island
24. Moved {'H3', 'W3'} from island to left
25. Moved {'W2', 'H2'} from left to island
26. 

In [14]:
solution = a_star(initial_state)

print("=== A* Solution ===")
if solution:
    for i, (people, frm, to) in enumerate(solution, 1):
        print(f"{i}. Moved {people} from {frm} to {to}")
    print(f"\nTotal steps: {len(solution)}")
else:
    print("No valid solution found.")


=== A* Solution ===
1. Moved {'W2', 'W1'} from left to island
2. Moved {'W2'} from island to left
3. Moved {'W2', 'W3'} from left to island
4. Moved {'W2'} from island to left
5. Moved {'H3', 'H1'} from left to island
6. Moved {'H3', 'W3'} from island to right
7. Moved {'H3'} from right to island
8. Moved {'H3'} from island to left
9. Moved {'H3', 'H2'} from left to island
10. Moved {'H3', 'H2'} from island to right
11. Moved {'H2'} from right to island
12. Moved {'W1'} from island to left
13. Moved {'W2', 'W1'} from left to island
14. Moved {'H1', 'H2'} from island to right
15. Moved {'W3'} from right to island
16. Moved {'W2', 'W1'} from island to right
17. Moved {'W2'} from right to island
18. Moved {'W2', 'W3'} from island to right

Total steps: 18
